In [90]:
import pandas as pd
import sqlite3 

In [91]:
connection = sqlite3.connect('../data/checking-logs.sqlite')

In [92]:
schema = pd.read_sql('PRAGMA table_info(test);', connection)
schema

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


In [93]:
pd.io.sql.read_sql('SELECT * FROM test LIMIT 10', connection)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


In [94]:
query = """
SELECT uid, MIN(cast((JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(dl.deadlines, 'unixepoch'))) * 24 AS INT)) AS min_diff
FROM test t
LEFT JOIN deadlines dl on t.labname = dl.labs
WHERE NOT t.labname = 'project1'
ORDER BY 2
LIMIT 1
"""

df_min = pd.io.sql.read_sql(query, connection)
df_min

,uid,min_diff
0,user_30,-202


In [95]:
query = """
SELECT uid, MAX(cast((JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(dl.deadlines, 'unixepoch'))) * 24 AS INT)) AS max_diff
FROM test t
LEFT JOIN deadlines dl on t.labname = dl.labs
WHERE NOT t.labname = 'project1'
ORDER BY 2
LIMIT 1
"""

df_max = pd.io.sql.read_sql(query, connection)
df_max

,uid,max_diff
0,user_25,-2


In [96]:
query = """
SELECT AVG(cast((JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(dl.deadlines, 'unixepoch'))) * 24 AS INT)) AS avg_diff
FROM test t
LEFT JOIN deadlines dl on t.labname = dl.labs
WHERE NOT t.labname = 'project1'
"""

df_avg = pd.io.sql.read_sql(query, connection)
df_avg

,avg_diff
0,-89.125


## Мы хотим проверить гипотезу о том, что пользователи, которые посетили новостную ленту всего несколько раз, имеют более низкую дельту между первым коммитом и дедлайном. Для этого нужно рассчитать коэффициент корреляции между количеством просмотров страниц и разницей

In [97]:
query = '''
SELECT
    t.uid,
    AVG(CAST((JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(d.deadlines, 'unixepoch'))) * 24 AS INT)) AS avg_diff,
    COUNT(pageviews.uid) as pageviews
FROM
    test t
LEFT JOIN 
    deadlines d ON t.labname = d.labs
LEFT JOIN
    pageviews ON t.uid = pageviews.uid
WHERE 
    t.labname != 'project1'
GROUP BY
    t.uid
'''
views_diff = pd.io.sql.read_sql(query, connection)
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,140
1,user_10,-74.800000,445
2,user_14,-159.000000,429
3,user_17,-61.600000,235
4,user_18,-5.666667,9
5,user_19,-98.750000,64
6,user_21,-95.500000,40
7,user_25,-92.600000,895
8,user_28,-86.400000,745
9,user_3,-105.400000,1585


In [98]:
views_diff.corr(numeric_only=True)

,avg_diff,pageviews
avg_diff,1.000000,-0.185834
pageviews,-0.185834,1.000000


In [99]:
connection.close()